In [ ]:
# Parse Dex JSON to CSV com One-hot encoding para arrays
import json
import pandas as pd

# Ler o JSON
with open('./pokedex-db.json', "r", encoding="utf-8") as f:
    pokedex_data = json.load(f)

# Converter para DataFrame
df = pd.DataFrame(pokedex_data)

# Garante que os campos de array estejam como listas (caso estejam ausentes)
for col in ['abilities', 'type', 'weakness']:
    df[col] = df[col].apply(lambda x: x if isinstance(x, list) else [])

# One-hot encoding para cada campo de array
abilities_dummies = df['abilities'].explode().str.strip().str.lower().str.title().dropna().unique()
type_dummies = df['type'].explode().str.strip().str.lower().str.title().dropna().unique()
weakness_dummies = df['weakness'].explode().str.strip().str.lower().str.title().dropna().unique()

df_abilities = df['abilities'].apply(lambda x: pd.Series({f'ability_{a}': int(a in x) for a in abilities_dummies}))
df_type = df['type'].apply(lambda x: pd.Series({f'type_{t}': int(t in x) for t in type_dummies}))
df_weakness = df['weakness'].apply(lambda x: pd.Series({f'weakness_{w}': int(w in x) for w in weakness_dummies}))

# Concatenar tudo
df_final = pd.concat([
    df.drop(['abilities', 'type', 'weakness'], axis=1),
    df_abilities,
    df_type,
    df_weakness
], axis=1)

# Salvar em CSV
df_final.to_csv('pokedex-db-parsed.csv', index=False, encoding='utf-8')

print("CSV com one-hot encoding gerado com sucesso!")

CSV com one-hot encoding gerado com sucesso!


In [ ]:
import requests
import time

def get_capture_locations(number):
    url = f"https://pokeapi.co/api/v2/pokemon/{number}/encounters"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            # Extrai o nome dos locais de captura
            locations = [loc['location_area']['name'] for loc in data]
            return locations
        else:
            return []
    except Exception as e:
        return []

# Adiciona a coluna de locais de captura (como lista)
df_final['capture_locations'] = df_final['number'].apply(get_capture_locations)

# Aguarda um pouco entre as requisições para não sobrecarregar a API
# (Opcional: descomente a linha abaixo se quiser ser mais cuidadoso)
time.sleep(0.2)

# Coleta todos os locais únicos
all_locations = set()
for locs in df_final['capture_locations']:
    all_locations.update(locs)
all_locations = sorted(all_locations)

# One-hot encoding dos locais de captura
for loc in all_locations:
    col_name = f'capture_{loc}'
    df_final[col_name] = df_final['capture_locations'].apply(lambda x: int(loc in x))

# (Opcional) Remova a coluna de lista se quiser só as one-hot
# df_final = df_final.drop(columns=['capture_locations'])

# Salva novamente o CSV com as novas colunas
df_final.to_csv('pokedex-db-with-regions.csv', index=False, encoding='utf-8')

print("Locais de captura adicionados e one-hot encoding realizado com sucesso!")

print(df_final.head())  # Exibe as primeiras linhas do DataFrame final